## **GIS 강의 2주차**

---

### **1.Geopandas 패키지 설치**
* 1-1) pip 로 Geopandas 설치시 나는 오루 
* 1-2) 해결방법 

### **2.파일 불러오기**
* 2-1) shp 파일 불러오기 
* 2-2) csv 파일 불러오기 

### **3.공간 데이터 핸들링**
* 3-1) 데이터 핸들링 
* 3-2) csv 파일의 x,y 컬럼을 공간 데이터(GeoDataframe) 로 변환 
* 3-3) 좌표계 변환하기 

### **4.Geopandas로 공간 속성 다루기**
* 4-1) 면적 (area) 
* 4-2) 길이
* 4-3) 도형의 무게중심
* 4-4) boundary 확인 
* 4-5) 공간의 유효성 확인 
* 4-6) 유효한 도형으로 만들기 

### **5.shapley 패키지 사용하여 shape 만들기**
* 5-1) 점(Point)
* 5-2) 선(LineString) 
* 5-3) 도형(Polygon) 
* 5-4) 한번에 여러 도형 만들기 (점,선,면))

### **6.공간데이터 활용하기**
* 6-1) 버퍼(buffer)
* 6-2) 도형을 감싸는 사각형 그리기(envelope) 
* 6-3) 도형 융해(dissolve)
* 6-4) 도형끼리 겹치는 부분 처리하기(overlay)

### **7.공간 데이터 결합**
* 7-1) 여러 공간 결합 방법들 
* 7-2) 결합 방법별 결과

### **8.저장하기**
* 8-1) shp 파일로 저장하기 

### **9.Reference
* 9-1) ref 

----


### **1.QGIS개요** 

* 무료 오픈소스 지리정보 시스템 Tool로서 공간정보 파일 시각화 및 여러 플러그인을 통하여 손쉽게 사용 가능 

* QGIS 다운로드 : [QGIS download](https://qgis.org/ko/site/forusers/download.html)

* QGiS 인터페이스 

<img width="667" alt="스크린샷 2022-09-05 오전 9 45 25" src="https://user-images.githubusercontent.com/83544197/188340429-f628ebdd-ff41-42be-9017-5155c3d60d65.png">

<br>

---